In [42]:
#naverfinance 에서 달러환율 가져오기 
from bs4 import BeautifulSoup 
import urllib.request as req

url= "https://finance.naver.com"

res = req.urlopen(url).read() # 바이너리 데이터로 처리하기 위한 작업 
data= res.decode("euc-kr") 

soup = BeautifulSoup(data,"html.parser")

price= soup.select_one("div.group1 > table.tbl_home > tbody > tr.up.bold > td").string
print(f"미국환율:{price}") 


미국환율:1,273.20


In [146]:
#"https://finance.naver.com" 에서 거래상위 10개를 추출하여 출력하세요 
# 종목 가격 -> 대한전기 2140 과 같은 식으로 출력 

from bs4 import BeautifulSoup 
import urllib.request as req

url= "https://finance.naver.com"

res = req.urlopen(url).read() # 바이너리 데이터로 처리하기 위한 작업 
data= res.decode("euc-kr") 
# **Page 내에 ID 값은 하나밖에 없기 때문에 ID값을 이용하느 것이 좋다. 

soup = BeautifulSoup(data,"html.parser") 

stocknames = soup.select("div.group_type.is_active > table.tbl_home > tbody#_topItems1 > tr.up > th > a  ")
stockprices = soup.select("div.group_type.is_active > table.tbl_home > tbody#_topItems1 > tr.up > td     ")

name= [] 
for stockname in stocknames:  
    name.append(stockname.string)
print(name)
# or
stocknames_list = [stocknames[i].string for i in range(10)]  
print(stocknames_list)


price=[] 
for stockprice in stockprices:   
    if stockprice.string is not None:
        price.append(stockprice.string)
print(price)
# or
stockprice_list= [stockprices[i].string for i in range(10)] 
print(stockprice_list)


print("[출력결과]")
for i in range (0,10): 
    print(f"{i+1}위: {name[i]}-{price[i].string}원" )
    

['대한전선', 'KODEX 200선물인버스2X', '부산주공', '이스트아시아홀딩스', '소니드', 'GS글로벌', '대원전선', '이루온', '로스웰', 'KODEX 인버스']
['대한전선', 'KODEX 200선물인버스2X', '부산주공', '이스트아시아홀딩스', '소니드', 'GS글로벌', '대원전선', '이루온', '로스웰', 'KODEX 인버스']
['2,320', '2,670', '885', '312', '1,355', '4,590', '2,305', '2,915', '417', '4,480']
['2,320', None, None, '2,670', None, None, '885', None, None, '312']
[출력결과]
1위: 대한전선-2,320원
2위: KODEX 200선물인버스2X-2,670원
3위: 부산주공-885원
4위: 이스트아시아홀딩스-312원
5위: 소니드-1,355원
6위: GS글로벌-4,590원
7위: 대원전선-2,305원
8위: 이루온-2,915원
9위: 로스웰-417원
10위: KODEX 인버스-4,480원


In [155]:
## 링크에 있는 내용을 한꺼번에 처리하기 
# 재귀적으로 탐색하기 위해 사용하는 함수: urljoin(base,"상대경로") 함수 
from urllib.parse import urljoin 

base = "http://example.com/html/a.html" 

## 상대경로
print(urljoin(base,"b.html"))  # base: 기본경로  
print(urljoin(base,"sub/c.html")) 
print(urljoin(base,"../index.html")) 
print(urljoin(base,"../img/home.png"))
print(urljoin(base,"../css/home.css")) # ../ 사용할 경우 com까지만 불러오겠다. 

## 절대경로
print(urljoin(base,"/index.html"))
print(urljoin(base,"http://otherExample.com/wiki")) # http:// 기존 base 경로에 새로운 URL을 덮어 씌운다. 
print(urljoin(base,"//otherExample.com/test"))



http://example.com/html/b.html
http://example.com/html/sub/c.html
http://example.com/index.html
http://example.com/img/home.png
http://example.com/css/home.css
http://example.com/index.html
http://otherExample.com/wiki
http://otherExample.com/test


In [ ]:
## 처리 순서 
# 1. HTML 분석 
# 2. 링크추출 
# 3. 각 링크 대상에 대해 다음동작을 구동함. 
#   3-1. 다운로드 파일인 경우 다운로드...
#   3-2. 파일이 HTML 이라면 1번으로 돌아가서 다시 실행





In [3]:
## 모든 페이지를 한꺼번에 다운받는 프로그렘을 만들기 (하기전에 사이트정보를 분석해야한다.)

## 모듈
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import * 
from os import makedirs
import os.path, time, re

## 처리한 내용이 파일인지 여부 확인을 위한 변수
proc_files = {}      # 딕셔너리 생성

# HTML 내부에 있는 링크를 추출하는 함수를 동작
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel ='stylesheet']")      ## CSS 참조
    links += soup.select("a[href]")     # 링크 참조
    
    result = []
    # href 속성을 추출하고, 링크를 절대 경로로 변환하여 저장
    for a in links:
        href = a.attrs['href'] # href 속성만 가져왔다.  
        url = urljoin(base, href)
        result.append(url)
    return result

# 파일을 다운로드하고 저장하는 함수
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path     ## netloc = http://를 제외한 도메인 주소 , path = 도메인 뒤에 있는 경로
    if re.search(r"/$", savepath):          # savepath의 마지막이 "/"라면
        savepath += "index.html"            # 기본 문서
    savedir = os.path.dirname(savepath)
    
    # 모두 다운로드됐는지 여부 확인
    if os.path.exists(savepath):
        return savepath
    
    # 다운로드 안됐을 시 다운받을 디렉터리를 생성
    if not os.path.exists(savedir):
        print("mkdir = ", savedir)
        makedirs(savedir)
    
    # 파일 다운받기
    try:
        print("download = ", url)
        urlretrieve(url,savepath)
        time.sleep(1)   # 1초 쉬었다가 동작
        return savepath
    except Exception as e:
        print("다운로드 오류 발생 : ", url)
        print("다운로드 에러 내용 : ", e)
        return None
    
# HTML을 분석하고 다운받는 함수
def analyze_html(url, root_url):
    savepath = download_file(url)
    if savepath is None:
        return
    if savepath in proc_files:      # 이미 처리되었다면 실행하지 않음
        return 
    proc_files[savepath] = True
    print("analyze_html=",url)
    
    # 링크 추출 
    html = open(savepath, "r", encoding="utf-8").read()
    links = enum_links(html, url)
    
    for link_url in links:
        # 링크가 루트 이외의 경로를 나타낸다면 무시
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url):
                continue
            
        # HTML이라면
        if re.search(r".(html|htm)$", link_url):
            # 재귀적으로 html 파일 분석
            analyze_html(link_url, root_url)
            continue
        # 기타 파일
        download_file(link_url)
        
# 일반 파이썬 파일인 경우 사용      
# if __name__ == "__main__":
#     # url에 있는 모든 것 다운받기
#     url = "https://docs.python.org/3.10/library/"
#     analyze_html(url, url)

url = "https://docs.python.org/3.10/library/"
analyze_html(url, url)

analyze_html= https://docs.python.org/3.10/library/
mkdir =  ./docs.python.org/3.10/_static
download =  https://docs.python.org/3.10/_static/pygments.css
download =  https://docs.python.org/3.10/library/intro.html
analyze_html= https://docs.python.org/3.10/library/intro.html
download =  https://docs.python.org/3.10/library/functions.html
analyze_html= https://docs.python.org/3.10/library/functions.html
download =  https://docs.python.org/3.10/library/constants.html
analyze_html= https://docs.python.org/3.10/library/constants.html
download =  https://docs.python.org/3.10/library/stdtypes.html
analyze_html= https://docs.python.org/3.10/library/stdtypes.html
download =  https://docs.python.org/3.10/library/exceptions.html
analyze_html= https://docs.python.org/3.10/library/exceptions.html
download =  https://docs.python.org/3.10/library/text.html
analyze_html= https://docs.python.org/3.10/library/text.html
download =  https://docs.python.org/3.10/library/string.html
analyze_html= https://d

In [158]:
from bs4 import BeautifulSoup 
from urllib.request import * 
from urllib.parse import* 
from os import makedirs 
import os.path, time, re 


url = "http://example.com/html/a.html" 
t = urlparse(url)#URL 주소경로정보를 파싱 
t.netloc #도메인 example.com
t.path #도메인 뒤의 경로 /html/a.html 

'/html/a.html'

In [ ]:
http.server -- directory 

In [ ]:
## requests 패키지를 이용한 접근   

In [5]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [9]:
# 파이썬으로 웹사이트 로그인 하기 
# 한빛출판사 로그인 화면 예제 
#<form name="frm"  id="frm"  action="#" method="post">
#   <input name="retun_url" id="retun_url" type="hidden" value="" class="i_text" size="100" >
	#<div class="login_left">		
		#<fieldset>
			#<legend>한빛출판네트워크 로그인</legend>
				
			#<label class="i_label" for="login_id"><strong></strong>
				#<input name="m_id" id="m_id" type="text" value="" class="i_text" placeholder="아이디" onkeydown="javascript:if(event.keyCode==13){login_proc(); return false;}">
			#</label> 

			#<label class="i_label" for="login_pw"><strong></strong>
				#<input name="m_passwd"  id="m_passwd" type="password" value="" class="i_text" placeholder="비밀번호" onkeydown="javascript:if(event.keyCode==13){login_proc(); return false;}">
			#</label>
				
			#<label>
				#<input  type="button" name="login_btn"  id="login_btn" value="로그인" class="btn_login" >					
			#</label>
				
			#<label class="i_label2">
				#<input type="checkbox" name="keepid" id="keepid" value="1" class="i_check"><strong>아이디 저장</strong>
			#</label>
		#</fieldset>
			
		#<ul class="login_btn">
			#<li><a href="https://www.hanbit.co.kr/member/find_id.html" class="btn_idc">아이디 찾기</a></li>
			#<li><a href="https://www.hanbit.co.kr/member/find_pw.html" class="btn_pwc">비밀번호 찾기</a></li>
			#<li><a href="https://www.hanbit.co.kr/member/member_agree.html" class="btn_joinc">회원가입</a></li>
		#</ul>
	#</div>
	#</form>
 
##파이썬으로 로그인하기 
import requests 
from bs4 import BeautifulSoup  
from urllib.parse import urljoin

#아이디와 비밀번호 지정 
USER = "wyh0913" 
PASS = "Qhpfrr132!" 

#세션시작 
session= requests.session() 

## 로그인하기 
login_info = {  
    "m_id":USER,
    "m_pw":PASS
}

url_login ="https://www.hanbit.co.kr/member/login.html"
res = session.post(url_login, data = login_info)
res.raise_for_status() #오류 발생 시 예외발생  

#마이페이지 접근 
url_mypage = "https://www.hanbit.co.kr/myhanbit/myhanbit.html"
res= session.get(url_mypage)
res.raise_for_status() 


soup = BeautifulSoup(res.text,"html.parser")
mileage = soup.select_one(".mileage_section1 span").get_text()
mileage 

#해당 예시는 사이트의 한계로 완성이 불가했지만 향후 Selenium 등을 활용해 구현이 가능 


AttributeError: 'NoneType' object has no attribute 'get_text'

In [14]:
## request 모듈의 메서드 

# request.get 요청
r = requests.get("http://naver.com")
r.content.decode("utf-8")

# request.post 요청 
formdata = {"key1":"value1", "key2":"value2"} 
r =requests.post("http://example.com", data = formdata) 
r.content



b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    

In [27]:
r= requests.put("http://httpbin.org/put",data="test") # URL 에 데이터를 업로드 하는기능
r= requests.delete("http://httpbin.org/delete") #파일을 삭제하는 기능
r= requests.head("http://httpbin.org/get")  #데이터를 제외한 서버로 부터 받은 Header 정보만 가져옴
#ex) 
rq_head= requests.head("https://www.python.org") 
print(rq_head)

# Get방식의 요청 vs 헤더방식의 요청 
print(requests.get("https://www.google.com").content)
rq_head= requests.head("https://www.python.org") 
print(rq_head)

ValueError: too many values to unpack (expected 2)

In [39]:
from selenium import webdriver 
USER = "wyh0913" 
PASS = "Qhpfrr132!" 

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path) 

url_login= "https://www.hanbit.co.kr/member/login.html" 
driver.get(url_login) 

m_id = driver.find_element_by_id('m_id') 
m_id.clear() 
m_id.send_keys(USER) 

m_passwd= driver.find_element_by_id('m_passwd') 
m_passwd.clear() 
m_passwd.send_keys(PASS) 

driver.find_element_by_class_name("btn_login").click() 
driver.get("https://www.hanbit.co.kr/myhanbit/myhanbit.html")


/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/884524501.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)
/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/884524501.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  m_id = driver.find_element_by_id('m_id')
/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/884524501.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  m_passwd= driver.find_element_by_id('m_passwd')
/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/884524501.py:18: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("btn_login").click()


In [38]:
mileage = driver.find_element_by_css_selector(".mileage_section1 span") 
ecoin = driver.find_element_by_css_selector(".mileage_section2 span ")
print(f"한빛 마일리지:{mileage.text}점 ")
print(f"한비 이코인:{ecoin.text}점")

한빛 마일리지:2,000점 
한비 이코인:0점


/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/330096161.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  mileage = driver.find_element_by_css_selector(".mileage_section1 span")
/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/330096161.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  ecoin = driver.find_element_by_css_selector(".mileage_section2 span ")


In [63]:
# 네이버 로그인 후 네이버페이 -> 네이버 포인트 가져오기
from selenium import webdriver 
USER = "wyh0913"
PASS = "Qhpfrr13##"

path = "/usr/local/bin/chromedriver" 
driver= webdriver.Chrome(path) 

url_login = "https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com"
driver.get(url_login)

id= driver.find_element_by_id("id") 
id.clear() 
id.send_keys(USER) 

pd = driver.find_element_by_id("pw")
pd.clear() 
pd.send_keys(PASS)

driver.find_element_by_class_name("btn_login").click() 
driver.get("https://order.pay.naver.com/home")

/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/2299826109.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver= webdriver.Chrome(path)
/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/2299826109.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  id= driver.find_element_by_id("id")
/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/2299826109.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  pd = driver.find_element_by_id("pw")
/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/2299826109.py:20: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  driver.find_element_by_class_name("btn_login").click()


In [60]:
mileage=driver.find_element_by_css_selector(".my_npoint strong")
print(f"나의 마일리지 포인트:{mileage.text}")

나의 마일리지 포인트:1,990


/var/folders/y7/jb39x55d38zfw6t2pdb72tw40000gn/T/ipykernel_15409/1920416560.py:1: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  mileage=driver.find_element_by_css_selector(".my_npoint strong")
